<a href="https://colab.research.google.com/github/ChihabEddine98/AmazonAnalysis/blob/main/AmazonAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [49.8 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 h

In [41]:
# imports
import os
import requests as rq
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.utils import ChromeType
import unicodedata
# end imports

# Part 01  : Data Gathering

In [42]:
# Settings 
chrome_driver_path = '/Users/chihab_eddine98/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver'

reviews_div_cls = 'review'

prod_urls = {'informatique':[
    {
        'prod_url':
        'https://www.amazon.fr/Microsoft-Surface-Pro-Hybride-stockage/dp/B07X8NL7MW/ref=sr_1_7? __mk_fr_FR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&dchild=1&keywords=Microsoft+Surface&qid=1616930888&  s=computers&sr=1-7',
    'rev_url': 'https://www.amazon.fr/Microsoft-Surface-Pro-Hybride-stockage/product-reviews/B07X8NL7MW/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews'
    
    }
    ]
}

ds_cols = ['Rev_Title','Rev_Rate','Rev_Bdy','Rev_Hlp','Prod_Title','Prod_Brnd','Prod_Rate','Prod_Eval','Prod_Price','Category']


# End Settings

# Scrapping Product Information

In [43]:

# Scrapping Product Data 
def product_scrapper(driver,prod_category,prod_url):
    driver.get(prod_url)
    prod_soup = BeautifulSoup(driver.page_source,'html.parser')

    # Product Info
    prod_title = prod_soup.find('span',{'id' : 'productTitle' }).text
    prod_brand = prod_soup.find('a',{'id' : 'bylineInfo' }).text
    prod_rate = prod_soup.find('span',{'class' : 'a-icon-alt' }).text
    prod_eval = prod_soup.find('span',{'id' : 'acrCustomerReviewText' }).text
    prod_price = prod_soup.find('span',{'id' : 'priceblock_ourprice' }).text


    result = tuple(map(lambda x : unicodedata.normalize("NFKD",x.strip()),[prod_title,prod_brand,       prod_rate,prod_eval,prod_price,prod_category]))
    return result



## Test Product Scrapping

In [45]:
driver = webdriver.Chrome()

for category,products in prod_urls.items():
    for prod in products:
        product = product_scrapper(driver,category,prod['prod_url'])
        print(product)

WebDriverException: ignored

# Scrapping Reviews Data

In [ ]:
# Scrapping Reviews Data
def reviews_scrapper(driver,prod,rev_url,ds):
    driver.get(rev_url)
    rev_soup = BeautifulSoup(driver.page_source,'html.parser')
    reviews_div = rev_soup.find_all('div',{'class':reviews_div_cls})

    for review in reviews_div:
        review_title = review.find('a',{'data-hook' : 'review-title'}).text
        review_rate = review.find('i',{'data-hook' : 'review-star-rating'}).text
        review_body = review.find('span',{'data-hook' : 'review-body'}).text
        review_help = review.find('span',{'data-hook' : 'helpful-vote-statement'}).text
        rev_result = tuple(map(lambda x : unicodedata.normalize("NFKD",x.strip()),[review_title,review_rate,review_body,review_help]))
        ds.append(rev_result+prod)

    return ds

## Test Reviews Scrapping

In [ ]:
ds = []
for category,products in prod_urls.items():
    for prod in products:
        product = product_scrapper(driver,category,prod['prod_url'])
        ds = reviews_scrapper(driver,product,prod['rev_url'],ds) 

df = pd.DataFrame(ds)
df.columns = ds_cols
df



